# ***Импорты***

In [1]:
%%capture
!pip install razdel
!pip install pyLDAvis
!pip install pymorphy2
!pip install dostoevsky
!python -m dostoevsky download fasttext-social-network-model
!pip install clean-text
from operator import itemgetter
from decimal import Decimal
from statistics import mean
from networkx.algorithms import community
import networkx.algorithms.community as nx_comm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *
from sklearn.model_selection import train_test_split 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import ensemble
from sklearn import svm
import time
from tqdm import tqdm
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel
from collections import Counter
import pandas as pd
import numpy as np
from wordcloud import WordCloud
from PIL import Image
from io import BytesIO
import networkx as nx
from razdel import sentenize
from razdel import tokenize
import nltk 
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('movie_reviews')
from nltk import ngrams
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import json
import requests
from urllib.request import urlopen
import string
from tqdm.notebook import tqdm
import gensim.corpora as corpora
from gensim.models import LdaMulticore, CoherenceModel
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis
import os
import matplotlib.pyplot as plt
import matplotlib
from pymorphy2 import MorphAnalyzer
from cleantext import clean
import warnings
import re
import sys
from matplotlib import pyplot
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import community
import networkx as nx
import pickle
import random
warnings.filterwarnings("ignore")

# ***Выгрузка датасета***

In [5]:
df = pd.read_csv("/content/finalSentimentdata2.csv")  
df = df.rename(columns = {'Unnamed: 0':' ID'})        
df                                                    

,ID,sentiment,text
0,3204,sad,agree the poor in india are treated badly thei...
1,1431,joy,if only i could have spent the with this cutie...
2,654,joy,will nature conservation remain a priority in ...
3,2530,sad,coronavirus disappearing in italy show this to...
4,2296,sad,uk records lowest daily virus death toll since...
...,...,...,...
3085,2579,sad,today at 02 30pm a 54 year old bangladeshi mal...
3086,3579,anger,corona virus i implore that you cease activity...
3087,221,joy,issa date once lockdown ends inshaallah (and c...
3088,2705,sad,the death toll due to covid 19 rose to 31 in j...


# ***Оставляем sentiment = joy и sentiment = sad***

In [6]:
data = df[(df["sentiment"] == "joy") | (df["sentiment"] == "sad")]  # Оставляем в датасете настроения "sad" и "joy"
data                                                                # Вывод нового датасета

,ID,sentiment,text
0,3204,sad,agree the poor in india are treated badly thei...
1,1431,joy,if only i could have spent the with this cutie...
2,654,joy,will nature conservation remain a priority in ...
3,2530,sad,coronavirus disappearing in italy show this to...
4,2296,sad,uk records lowest daily virus death toll since...
...,...,...,...
3083,2194,joy,it was tough to see you go brother excellent 6...
3085,2579,sad,today at 02 30pm a 54 year old bangladeshi mal...
3087,221,joy,issa date once lockdown ends inshaallah (and c...
3088,2705,sad,the death toll due to covid 19 rose to 31 in j...


In [7]:
data.sentiment.unique()  # Проверка правильности запроса

array(['sad', 'joy'], dtype=object)

# ***Перевод текста в нижний регистр***

In [8]:
lower_regist = data['text'].str.lower()  
data['lower'] = lower_regist             
data                   

,ID,sentiment,text,lower
0,3204,sad,agree the poor in india are treated badly thei...,agree the poor in india are treated badly thei...
1,1431,joy,if only i could have spent the with this cutie...,if only i could have spent the with this cutie...
2,654,joy,will nature conservation remain a priority in ...,will nature conservation remain a priority in ...
3,2530,sad,coronavirus disappearing in italy show this to...,coronavirus disappearing in italy show this to...
4,2296,sad,uk records lowest daily virus death toll since...,uk records lowest daily virus death toll since...
...,...,...,...,...
3083,2194,joy,it was tough to see you go brother excellent 6...,it was tough to see you go brother excellent 6...
3085,2579,sad,today at 02 30pm a 54 year old bangladeshi mal...,today at 02 30pm a 54 year old bangladeshi mal...
3087,221,joy,issa date once lockdown ends inshaallah (and c...,issa date once lockdown ends inshaallah (and c...
3088,2705,sad,the death toll due to covid 19 rose to 31 in j...,the death toll due to covid 19 rose to 31 in j...


# ***Токенизация***

In [9]:
tokenization = data.apply(lambda row: nltk.word_tokenize(row['lower']), axis=1)  
data['token'] = tokenization                                                    
data

,ID,sentiment,text,lower,token
0,3204,sad,agree the poor in india are treated badly thei...,agree the poor in india are treated badly thei...,"[agree, the, poor, in, india, are, treated, ba..."
1,1431,joy,if only i could have spent the with this cutie...,if only i could have spent the with this cutie...,"[if, only, i, could, have, spent, the, with, t..."
2,654,joy,will nature conservation remain a priority in ...,will nature conservation remain a priority in ...,"[will, nature, conservation, remain, a, priori..."
3,2530,sad,coronavirus disappearing in italy show this to...,coronavirus disappearing in italy show this to...,"[coronavirus, disappearing, in, italy, show, t..."
4,2296,sad,uk records lowest daily virus death toll since...,uk records lowest daily virus death toll since...,"[uk, records, lowest, daily, virus, death, tol..."
...,...,...,...,...,...
3083,2194,joy,it was tough to see you go brother excellent 6...,it was tough to see you go brother excellent 6...,"[it, was, tough, to, see, you, go, brother, ex..."
3085,2579,sad,today at 02 30pm a 54 year old bangladeshi mal...,today at 02 30pm a 54 year old bangladeshi mal...,"[today, at, 02, 30pm, a, 54, year, old, bangla..."
3087,221,joy,issa date once lockdown ends inshaallah (and c...,issa date once lockdown ends inshaallah (and c...,"[issa, date, once, lockdown, ends, inshaallah,..."
3088,2705,sad,the death toll due to covid 19 rose to 31 in j...,the death toll due to covid 19 rose to 31 in j...,"[the, death, toll, due, to, covid, 19, rose, t..."


# ***Удаление стоп-слов***

In [10]:
stop_word = stopwords.words('english')                                                  
no_stop = data["token"].apply(lambda x: [item for item in x if item not in stop_word])  
data['no_stop_words'] = no_stop                                                         
data                                                                                   

,ID,sentiment,text,lower,token,no_stop_words
0,3204,sad,agree the poor in india are treated badly thei...,agree the poor in india are treated badly thei...,"[agree, the, poor, in, india, are, treated, ba...","[agree, poor, india, treated, badly, poors, se..."
1,1431,joy,if only i could have spent the with this cutie...,if only i could have spent the with this cutie...,"[if, only, i, could, have, spent, the, with, t...","[could, spent, cutie, vc, sakshi__s, n, g, h, ..."
2,654,joy,will nature conservation remain a priority in ...,will nature conservation remain a priority in ...,"[will, nature, conservation, remain, a, priori...","[nature, conservation, remain, priority, post,..."
3,2530,sad,coronavirus disappearing in italy show this to...,coronavirus disappearing in italy show this to...,"[coronavirus, disappearing, in, italy, show, t...","[coronavirus, disappearing, italy, show, ``, i..."
4,2296,sad,uk records lowest daily virus death toll since...,uk records lowest daily virus death toll since...,"[uk, records, lowest, daily, virus, death, tol...","[uk, records, lowest, daily, virus, death, tol..."
...,...,...,...,...,...,...
3083,2194,joy,it was tough to see you go brother excellent 6...,it was tough to see you go brother excellent 6...,"[it, was, tough, to, see, you, go, brother, ex...","[tough, see, go, brother, excellent, 60, days,..."
3085,2579,sad,today at 02 30pm a 54 year old bangladeshi mal...,today at 02 30pm a 54 year old bangladeshi mal...,"[today, at, 02, 30pm, a, 54, year, old, bangla...","[today, 02, 30pm, 54, year, old, bangladeshi, ..."
3087,221,joy,issa date once lockdown ends inshaallah (and c...,issa date once lockdown ends inshaallah (and c...,"[issa, date, once, lockdown, ends, inshaallah,...","[issa, date, lockdown, ends, inshaallah, (, co..."
3088,2705,sad,the death toll due to covid 19 rose to 31 in j...,the death toll due to covid 19 rose to 31 in j...,"[the, death, toll, due, to, covid, 19, rose, t...","[death, toll, due, covid, 19, rose, 31, jammu,..."


# ***Лемматизация***

In [11]:
lemmatizer = WordNetLemmatizer()                                                                 # Создание экземпляра лемматизатора WordNetLemmatizer
lemm_review = data["no_stop_words"].apply(lambda x: [lemmatizer.lemmatize(word) for word in x]) 
data['lemmatization'] = lemm_review                                                            
data                                                                                           

,ID,sentiment,text,lower,token,no_stop_words,lemmatization
0,3204,sad,agree the poor in india are treated badly thei...,agree the poor in india are treated badly thei...,"[agree, the, poor, in, india, are, treated, ba...","[agree, poor, india, treated, badly, poors, se...","[agree, poor, india, treated, badly, poor, see..."
1,1431,joy,if only i could have spent the with this cutie...,if only i could have spent the with this cutie...,"[if, only, i, could, have, spent, the, with, t...","[could, spent, cutie, vc, sakshi__s, n, g, h, ...","[could, spent, cutie, vc, sakshi__s, n, g, h, ..."
2,654,joy,will nature conservation remain a priority in ...,will nature conservation remain a priority in ...,"[will, nature, conservation, remain, a, priori...","[nature, conservation, remain, priority, post,...","[nature, conservation, remain, priority, post,..."
3,2530,sad,coronavirus disappearing in italy show this to...,coronavirus disappearing in italy show this to...,"[coronavirus, disappearing, in, italy, show, t...","[coronavirus, disappearing, italy, show, ``, i...","[coronavirus, disappearing, italy, show, ``, i..."
4,2296,sad,uk records lowest daily virus death toll since...,uk records lowest daily virus death toll since...,"[uk, records, lowest, daily, virus, death, tol...","[uk, records, lowest, daily, virus, death, tol...","[uk, record, lowest, daily, virus, death, toll..."
...,...,...,...,...,...,...,...
3083,2194,joy,it was tough to see you go brother excellent 6...,it was tough to see you go brother excellent 6...,"[it, was, tough, to, see, you, go, brother, ex...","[tough, see, go, brother, excellent, 60, days,...","[tough, see, go, brother, excellent, 60, day, ..."
3085,2579,sad,today at 02 30pm a 54 year old bangladeshi mal...,today at 02 30pm a 54 year old bangladeshi mal...,"[today, at, 02, 30pm, a, 54, year, old, bangla...","[today, 02, 30pm, 54, year, old, bangladeshi, ...","[today, 02, 30pm, 54, year, old, bangladeshi, ..."
3087,221,joy,issa date once lockdown ends inshaallah (and c...,issa date once lockdown ends inshaallah (and c...,"[issa, date, once, lockdown, ends, inshaallah,...","[issa, date, lockdown, ends, inshaallah, (, co...","[issa, date, lockdown, end, inshaallah, (, cor..."
3088,2705,sad,the death toll due to covid 19 rose to 31 in j...,the death toll due to covid 19 rose to 31 in j...,"[the, death, toll, due, to, covid, 19, rose, t...","[death, toll, due, covid, 19, rose, 31, jammu,...","[death, toll, due, covid, 19, rose, 31, jammu,..."


# ***Разделение данных на тестовые и тренировочные***

In [12]:
lemma_list = data.lemmatization   # Создание списка лемматизированных токенов
no_list = []                   
for i in lemma_list:            
    no_list.append(", ".join(i))
data['lemma_no_list'] = no_list   # Создание нового столбца для лемматизированных токенов, которые выведены строкой, а не списком
data

,ID,sentiment,text,lower,token,no_stop_words,lemmatization,lemma_no_list
0,3204,sad,agree the poor in india are treated badly thei...,agree the poor in india are treated badly thei...,"[agree, the, poor, in, india, are, treated, ba...","[agree, poor, india, treated, badly, poors, se...","[agree, poor, india, treated, badly, poor, see...","agree, poor, india, treated, badly, poor, seek..."
1,1431,joy,if only i could have spent the with this cutie...,if only i could have spent the with this cutie...,"[if, only, i, could, have, spent, the, with, t...","[could, spent, cutie, vc, sakshi__s, n, g, h, ...","[could, spent, cutie, vc, sakshi__s, n, g, h, ...","could, spent, cutie, vc, sakshi__s, n, g, h, c..."
2,654,joy,will nature conservation remain a priority in ...,will nature conservation remain a priority in ...,"[will, nature, conservation, remain, a, priori...","[nature, conservation, remain, priority, post,...","[nature, conservation, remain, priority, post,...","nature, conservation, remain, priority, post, ..."
3,2530,sad,coronavirus disappearing in italy show this to...,coronavirus disappearing in italy show this to...,"[coronavirus, disappearing, in, italy, show, t...","[coronavirus, disappearing, italy, show, ``, i...","[coronavirus, disappearing, italy, show, ``, i...","coronavirus, disappearing, italy, show, ``, in..."
4,2296,sad,uk records lowest daily virus death toll since...,uk records lowest daily virus death toll since...,"[uk, records, lowest, daily, virus, death, tol...","[uk, records, lowest, daily, virus, death, tol...","[uk, record, lowest, daily, virus, death, toll...","uk, record, lowest, daily, virus, death, toll,..."
...,...,...,...,...,...,...,...,...
3083,2194,joy,it was tough to see you go brother excellent 6...,it was tough to see you go brother excellent 6...,"[it, was, tough, to, see, you, go, brother, ex...","[tough, see, go, brother, excellent, 60, days,...","[tough, see, go, brother, excellent, 60, day, ...","tough, see, go, brother, excellent, 60, day, t..."
3085,2579,sad,today at 02 30pm a 54 year old bangladeshi mal...,today at 02 30pm a 54 year old bangladeshi mal...,"[today, at, 02, 30pm, a, 54, year, old, bangla...","[today, 02, 30pm, 54, year, old, bangladeshi, ...","[today, 02, 30pm, 54, year, old, bangladeshi, ...","today, 02, 30pm, 54, year, old, bangladeshi, m..."
3087,221,joy,issa date once lockdown ends inshaallah (and c...,issa date once lockdown ends inshaallah (and c...,"[issa, date, once, lockdown, ends, inshaallah,...","[issa, date, lockdown, ends, inshaallah, (, co...","[issa, date, lockdown, end, inshaallah, (, cor...","issa, date, lockdown, end, inshaallah, (, coro..."
3088,2705,sad,the death toll due to covid 19 rose to 31 in j...,the death toll due to covid 19 rose to 31 in j...,"[the, death, toll, due, to, covid, 19, rose, t...","[death, toll, due, covid, 19, rose, 31, jammu,...","[death, toll, due, covid, 19, rose, 31, jammu,...","death, toll, due, covid, 19, rose, 31, jammu, ..."


In [13]:
x_train, x_test, y_train, y_test = train_test_split(data.lemma_no_list, data.sentiment, train_size = 0.8)  # Разделение данных на тренировочные (80%) и тестовые (20%)

# ***Векторизация***

In [14]:
vectorizer = CountVectorizer(ngram_range=(1, 3))        # Создание экземляра векторизатора CountVectorizer для одного слова, для пар слов и для троек слов
vectorized_x_train = vectorizer.fit_transform(x_train)  # Векторизуем тренировочные данные
vectorized_x_test = vectorizer.transform(x_test)        # Векторизуем тестовые данные

In [15]:
vectorized_x_train 

<1217x35283 sparse matrix of type '<class 'numpy.int64'>'
	with 51367 stored elements in Compressed Sparse Row format>

In [16]:
vectorized_x_test

<305x35283 sparse matrix of type '<class 'numpy.int64'>'
	with 5232 stored elements in Compressed Sparse Row format>

# ***Обучение многоклассового Баейсовского наивного классификатора***

https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

In [17]:
clf = MultinomialNB(force_alpha=True, alpha=0)                      # Импорт классификатора
clf.fit(vectorized_x_train, y_train)                                # Обучение классификатора на векторизованной тренировочной выборке
pred = clf.predict(vectorized_x_test)                               # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred))
print(classification_report(y_test, pred))                         

Точность предсказаний равна 0.6819672131147541
              precision    recall  f1-score   support

         joy       0.62      0.90      0.74       150
         sad       0.83      0.47      0.60       155

    accuracy                           0.68       305
   macro avg       0.73      0.69      0.67       305
weighted avg       0.73      0.68      0.67       305



## ***Попытка улучшения результатов***

In [18]:
clf = MultinomialNB(force_alpha=False, alpha=1)                     # Импорт нового классификатора с включекнным сглаживанием Лапласса
clf.fit(vectorized_x_train, y_train)                                # Обучение классификатора на векторизованной тренировочной выборке
pred = clf.predict(vectorized_x_test)                               # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred))  
print(classification_report(y_test, pred))                          

Точность предсказаний равна 0.8360655737704918
              precision    recall  f1-score   support

         joy       0.90      0.75      0.82       150
         sad       0.79      0.92      0.85       155

    accuracy                           0.84       305
   macro avg       0.85      0.83      0.83       305
weighted avg       0.85      0.84      0.83       305



# ***Обучение классификатора алгоритма случайного леса***

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [19]:
clf1 = ensemble.RandomForestClassifier(n_estimators=1522, random_state=0, criterion="gini")  # Импорт классификатора с количеством деревьев, равным количеству строк в датасете
clf1.fit(vectorized_x_train, y_train)                                                        # Обучение классификатора на векторизованной тренировочной выборке
pred1 = clf1.predict(vectorized_x_test)                                                      # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred1))                         
print(classification_report(y_test, pred1))                     

Точность предсказаний равна 0.819672131147541
              precision    recall  f1-score   support

         joy       0.75      0.95      0.84       150
         sad       0.93      0.70      0.80       155

    accuracy                           0.82       305
   macro avg       0.84      0.82      0.82       305
weighted avg       0.84      0.82      0.82       305



## ***Попытка улучшения результатов***

In [20]:
clf1 = ensemble.RandomForestClassifier(n_estimators=100, random_state=0, criterion="gini")   # Импорт нового классификатора с количеством деревьев, равным 100
clf1.fit(vectorized_x_train, y_train)                                                        # Обучение классификатора на векторизованной тренировочной выборке
pred1 = clf1.predict(vectorized_x_test)                                                      # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred1))                       
print(classification_report(y_test, pred1))                                              

Точность предсказаний равна 0.8065573770491803
              precision    recall  f1-score   support

         joy       0.74      0.94      0.83       150
         sad       0.92      0.68      0.78       155

    accuracy                           0.81       305
   macro avg       0.83      0.81      0.80       305
weighted avg       0.83      0.81      0.80       305



In [21]:
clf1 = ensemble.RandomForestClassifier(n_estimators=100, random_state=0, criterion="entropy")  # Импорт нового классификатора с энтропией в качестве функции для измерения качества разделения
clf1.fit(vectorized_x_train, y_train)                                                          # Обучение классификатора на векторизованной тренировочной выборке
pred1 = clf1.predict(vectorized_x_test)                                                        # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred1))                     
print(classification_report(y_test, pred1))                                             

Точность предсказаний равна 0.8262295081967214
              precision    recall  f1-score   support

         joy       0.75      0.97      0.85       150
         sad       0.96      0.69      0.80       155

    accuracy                           0.83       305
   macro avg       0.85      0.83      0.82       305
weighted avg       0.85      0.83      0.82       305



In [22]:
clf1 = ensemble.RandomForestClassifier(n_estimators=1500, random_state=0, criterion="gini")  # Импорт классификатора с количеством деревьев, равным количеству строк в датасете, и 
clf1.fit(vectorized_x_train, y_train)                                                           # Обучение классификатора на векторизованной тренировочной выборке
pred1 = clf1.predict(vectorized_x_test)                                                         # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred1))                           
print(classification_report(y_test, pred1))                                                   

Точность предсказаний равна 0.8229508196721311
              precision    recall  f1-score   support

         joy       0.75      0.95      0.84       150
         sad       0.94      0.70      0.80       155

    accuracy                           0.82       305
   macro avg       0.85      0.83      0.82       305
weighted avg       0.85      0.82      0.82       305



# ***Обучение классификатора метода опорных векторов***

https://scikit-learn.org/stable/modules/svm.html

In [23]:
clf2 = svm.SVC()                                                     # Импорт классификатора метода опорных векторов
clf2.fit(vectorized_x_train, y_train)                                # Обучение классификатора на векторизованной тренировочной выборке
pred2 = clf2.predict(vectorized_x_test)                              # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred2)) 
print(classification_report(y_test, pred2))                         

Точность предсказаний равна 0.7540983606557377
              precision    recall  f1-score   support

         joy       0.67      0.99      0.80       150
         sad       0.98      0.53      0.69       155

    accuracy                           0.75       305
   macro avg       0.82      0.76      0.74       305
weighted avg       0.83      0.75      0.74       305



## ***Попытка улучшения результатов***

In [24]:
clf2 = svm.LinearSVC()                                               # Импорт нового классификатора линейного метода опорных векторов
clf2.fit(vectorized_x_train, y_train)                                # Обучение классификатора на векторизованной тренировочной выборке
pred2 = clf2.predict(vectorized_x_test)                              # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred2)) 
print(classification_report(y_test, pred2))                         

Точность предсказаний равна 0.8229508196721311
              precision    recall  f1-score   support

         joy       0.78      0.89      0.83       150
         sad       0.88      0.75      0.81       155

    accuracy                           0.82       305
   macro avg       0.83      0.82      0.82       305
weighted avg       0.83      0.82      0.82       305



# ***Обучение классификатора метода ближайших соседей***

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

In [25]:
clf3 = KNeighborsClassifier(n_neighbors=5)                           # Импорт классификатора с количеством соседей, равным 5
clf3.fit(vectorized_x_train, y_train)                                # Обучение классификатора на векторизованной тренировочной выборке
pred3 = clf3.predict(vectorized_x_test)                              # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred3)) 
print(classification_report(y_test, pred3))                         

Точность предсказаний равна 0.49836065573770494
              precision    recall  f1-score   support

         joy       0.49      0.98      0.66       150
         sad       0.62      0.03      0.06       155

    accuracy                           0.50       305
   macro avg       0.56      0.51      0.36       305
weighted avg       0.56      0.50      0.35       305



## ***Попытка улучшения результатов***

In [26]:
clf3 = KNeighborsClassifier(n_neighbors=3)                           # Импорт классификатора с количеством соседей, равным 3
clf3.fit(vectorized_x_train, y_train)                                # Обучение классификатора на векторизованной тренировочной выборке
pred3 = clf3.predict(vectorized_x_test)                              # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred3)) 
print(classification_report(y_test, pred3))                         

Точность предсказаний равна 0.5245901639344263
              precision    recall  f1-score   support

         joy       0.51      0.89      0.65       150
         sad       0.61      0.17      0.27       155

    accuracy                           0.52       305
   macro avg       0.56      0.53      0.46       305
weighted avg       0.56      0.52      0.46       305



In [27]:
clf3 = KNeighborsClassifier(n_neighbors=7)                           # Импорт классификатора с количеством соседей, равным 7
clf3.fit(vectorized_x_train, y_train)                                # Обучение классификатора на векторизованной тренировочной выборке
pred3 = clf3.predict(vectorized_x_test)                              # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred3)) 
print(classification_report(y_test, pred3))                         

Точность предсказаний равна 0.49508196721311476
              precision    recall  f1-score   support

         joy       0.49      0.99      0.66       150
         sad       0.60      0.02      0.04       155

    accuracy                           0.50       305
   macro avg       0.55      0.50      0.35       305
weighted avg       0.55      0.50      0.34       305



In [28]:
clf3 = KNeighborsClassifier(n_neighbors=9)                           # Импорт классификатора с количеством соседей, равным 9
clf3.fit(vectorized_x_train, y_train)                                # Обучение классификатора на векторизованной тренировочной выборке
pred3 = clf3.predict(vectorized_x_test)                              # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred3))  
print(classification_report(y_test, pred3))                          

Точность предсказаний равна 0.5016393442622951
              precision    recall  f1-score   support

         joy       0.50      1.00      0.66       150
         sad       1.00      0.02      0.04       155

    accuracy                           0.50       305
   macro avg       0.75      0.51      0.35       305
weighted avg       0.75      0.50      0.35       305



# ***Обучение классификатора метода градиентного бустинга***

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html

In [29]:
clf4 = ensemble.GradientBoostingClassifier()                         # Импорт классификатора
clf4.fit(vectorized_x_train, y_train)                                # Обучение классификатора на векторизованной тренировочной выборке
pred4 = clf4.predict(vectorized_x_test)                              # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred4))  
print(classification_report(y_test, pred4))                         

Точность предсказаний равна 0.819672131147541
              precision    recall  f1-score   support

         joy       0.77      0.91      0.83       150
         sad       0.90      0.73      0.80       155

    accuracy                           0.82       305
   macro avg       0.83      0.82      0.82       305
weighted avg       0.83      0.82      0.82       305



## ***Попытка улучшения результатов***

In [30]:
accuracy = 0
for i in np.arange(0.0, 1.05, 0.05):
  clf4 = ensemble.GradientBoostingClassifier(learning_rate=i, random_state=0)
  clf4.fit(vectorized_x_train, y_train) 
  pred4 = clf4.predict(vectorized_x_test)
  new_accuracy = accuracy_score(y_test, pred4)
  if new_accuracy > accuracy:
    accuracy = new_accuracy
    state = i
  
print(accuracy)
print(state)

0.8459016393442623
0.30000000000000004


In [31]:
clf4 = ensemble.GradientBoostingClassifier(learning_rate=0.25, random_state=0)  # Импорт нового классификатора со скоростью обучения и заданным рандомным состоянием
clf4.fit(vectorized_x_train, y_train)                                           # Обучение классификатора на векторизованной тренировочной выборке
pred4 = clf4.predict(vectorized_x_test)                                         # Получение предсказаний для векторизованной тестовый выборки 
print("Точность предсказаний равна", accuracy_score(y_test, pred4))            
print(classification_report(y_test, pred4))                                    

Точность предсказаний равна 0.8360655737704918
              precision    recall  f1-score   support

         joy       0.79      0.91      0.84       150
         sad       0.89      0.77      0.83       155

    accuracy                           0.84       305
   macro avg       0.84      0.84      0.84       305
weighted avg       0.84      0.84      0.84       305



# ***Предобработка для второго датасета***

In [33]:
covid = pd.read_csv("/content/covid19_tweets.csv")
covid

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624.0,950.0,18775.0,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253.0,1677.0,24.0,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275.0,9525.0,7254.0,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197.0,987.0,1488.0,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009.0,168.0,101.0,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2760,HelloHealthy,NaN,NaN,2012-11-23 05:38:37,951.0,291.0,155.0,False,2020-07-25 10:49:03,#MadhyaPradesh CM #ChouhanShivraj tests positi...,"['MadhyaPradesh', 'ChouhanShivraj', 'COVID19',...",Twitter for Android,False
2761,The Hawk,India,▌Stay UpToDate - The Hawk is One of India's to...,2012-11-19 14:48:20,2088.0,802.0,14.0,False,2020-07-25 10:49:03,Maha: COVID-19 hospital overcharges patients; ...,"['COVID19', 'Maharashtra']",Hocalwire Social Share,False
2762,Eyewitness News,South Africa,INSIDE THE TRUTH,2009-02-09 13:03:22,1164651.0,3706.0,268.0,True,2020-07-25 10:49:02,Over 160k jobs stand to be lost in @WesternCap...,['COVID19'],dlvr.it,False
2763,VH Boutique,"Crawfordville, FL",Fabulous Makeup and more!\nDirect message me t...,2014-01-25 02:13:14,175.0,208.0,909.0,False,2020-07-25 10:48:59,You need to hear this story! I believe masks d...,"['WearAMask', 'COVID19']",Twitter for Android,False


In [34]:
covid = covid[["text"]]
covid

,text
0,If I smelled the scent of hand sanitizers toda...
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...
3,@brookbanktv The one gift #COVID19 has give me...
4,25 July : Media Bulletin on Novel #CoronaVirus...
...,...
2760,#MadhyaPradesh CM #ChouhanShivraj tests positi...
2761,Maha: COVID-19 hospital overcharges patients; ...
2762,Over 160k jobs stand to be lost in @WesternCap...
2763,You need to hear this story! I believe masks d...


In [35]:
def remove_emoji(string):                                                       # Параметры для запуска функции: строка (в нашем случае это текст записи)                                                
  """Функция для очистки строки от эмодзи"""
  emoji_pattern = re.compile("[" u"\U0001F600-\U0001F64F"                       # Сбор всех возможных смайликов и эмодзи  
                               u"\U0001F300-\U0001F5FF"  
                               u"\U0001F680-\U0001F6FF"  
                               u"\U0001F1E0-\U0001F1FF"  
                               u"\U00002500-\U00002BEF"  
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f" 
                               u"\u3030"
                               u"\u23F0" "]+", flags=re.UNICODE)
  return emoji_pattern.sub(r'', string)                                         # Возврат строки без смайликов

In [43]:
cleaned_texts = []                                                 # пустой список, в который поместим очищенные тексты
stop_word = stopwords.words('english')                             
prefix = ('@', '#', 'http', 'https', '//', "n't", "'s")            # Префиксы для удаления
for i in range(covid.shape[0]):                                    # Создаём цикл, который проходит по каждому тексту из датасета
  s = ''                                                           
  a = 0                                                            # Текущее слово из текста (изначально равно нулю)
  text = str(covid.text[i])
  text = text.replace('…', '')
  text = text.replace('\n', ' ')
  text = text.replace('&amp', ' ')
  text = text.lower()                                              
  text = remove_emoji(text)                                        # Убираем из текста смайлики и эмоджи с помощью написанной выше функции
  text = word_tokenize(text)                                       # Проводим токенизацию
  text = [word for word in text if not word in stop_word]          # Удаляем стоп-слова
  text = [lemmatizer.lemmatize(word) for word in text]             # Проводим лемматизацию
  while a < len(text) - 1:                                         # Создаём цикл while, который проходит по каждому слову в тексте
    if not text[a].startswith(prefix):                             # Если слово начинается не с тех символов, которые лежат в переменной prefix, то
      if a != len(text) - 1:                                       # если рассматриваемое слово не последнее, то
        s += text[a] + ', '                                        # добавляем его в строку и ставим после него запятую
      else:                                                        # Если же рассматриваемое слово последнее, то
        s += text[a]                                               # просто добавляем его в строку
      a += 1                                                       # Переходим к следующему слову                                                                           
    else:                                                          # Если слово начинается с символов, которые лежат в переменной prefix, то
      if text[a].startswith('#') or text[a].startswith('@'):       # если слово начинается с символа '@' или '#', то
        a += 2                                                     # пропускаем это и следующее слово
      else:                                                        # Если же слово начинается не с символов '@' или '#', то
        a += 1                                                     # пропускаем это слово
  s = re.sub(r'[^\w\s]', '', s)                                    # удаление знаков препинания
  s = ', '.join(s.split())                                        
  cleaned_texts.append(s)                                         

In [44]:
covid["full_clear"] = cleaned_texts
covid

,text,full_clear
0,If I smelled the scent of hand sanitizers toda...,"smelled, scent, hand, sanitizers, today, someo..."
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,"hey, would, made, sense, player, pay, respect"
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,"trump, never, claimed, hoax, claim, effort"
3,@brookbanktv The one gift #COVID19 has give me...,"one, gift, give, appreciation, simple, thing, ..."
4,25 July : Media Bulletin on Novel #CoronaVirus...,"25, july, medium, bulletin, novel"
...,...,...
2760,#MadhyaPradesh CM #ChouhanShivraj tests positi...,"cm, test, positive"
2761,Maha: COVID-19 hospital overcharges patients; ...,"maha, covid19, hospital, overcharge, patient, ..."
2762,Over 160k jobs stand to be lost in @WesternCap...,"160k, job, stand, lost, due"
2763,You need to hear this story! I believe masks d...,"need, hear, story, believe, mask, help, u, sta..."


In [ ]:
symbols = []                                                                                        # Создаём пустой список для символов
for i in range(covid.shape[0]):                                                                     # Создаём цикл, который проходит по каждому предобработанному тексту
  text = word_tokenize(covid.full_clear[i])                                                         # Проводим токенизацию
  for j in range(len(text)):                                                                        # Создаём цикл, который проходит по каждому токену
    if (len(text[j]) == 1 or len(text[j]) == 2) and text[j] != ',' and text[j].isdigit() == False:  # Если длина токена равна 1 или 2, а также он не является запятой и не содержит число, то
      symbols.append([i, text[j]])                                                                  # Добавляем номер текста и символ в список
symbols                                                                                             # Выводим список

In [46]:
stats = []                                     # Создаём пустой список
for i in range(covid.shape[0]):                # Создаём цикл, который проходит по всем предобработанным текстам
  stats.append([i, len(covid.full_clear[i])])  # Добавляем в список номер текста и его длину
stats = sorted(stats, key=itemgetter(1))       # Отсортировываем список по длине текста

In [47]:
statistics = pd.DataFrame(stats)                                                                         # Создаём датафрейм из списка
statistics.rename(columns={statistics.columns[0]: 'ID', statistics.columns[1]: 'length'}, inplace=True)  # Переименовываем столбцы датафрейма
statistics                                                                                               # Выводим датафрейм

,ID,length
0,113,0
1,264,0
2,312,0
3,980,0
4,1083,0
...,...,...
2760,2330,117
2761,1365,118
2762,2624,119
2763,1273,120


## Результат предобработки второго текста

In [48]:
final = covid[covid['full_clear'].str.contains(',')]  # Оставляем только те тексты, в которых имеется как минимум два слова (то есть строка содержит запятую)
final                                                 

,text,full_clear
0,If I smelled the scent of hand sanitizers toda...,"smelled, scent, hand, sanitizers, today, someo..."
1,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,"hey, would, made, sense, player, pay, respect"
2,@diane3443 @wdunlap @realDonaldTrump Trump nev...,"trump, never, claimed, hoax, claim, effort"
3,@brookbanktv The one gift #COVID19 has give me...,"one, gift, give, appreciation, simple, thing, ..."
4,25 July : Media Bulletin on Novel #CoronaVirus...,"25, july, medium, bulletin, novel"
...,...,...
2759,Where are people going to on holidays? Eimer H...,"people, going, holiday, eimer, hannon, talk, e..."
2760,#MadhyaPradesh CM #ChouhanShivraj tests positi...,"cm, test, positive"
2761,Maha: COVID-19 hospital overcharges patients; ...,"maha, covid19, hospital, overcharge, patient, ..."
2762,Over 160k jobs stand to be lost in @WesternCap...,"160k, job, stand, lost, due"


# ***Предсказание для нового датасета с помощью обученной на старых данных модели***

In [49]:
vectorized_text = vectorizer.transform(final.full_clear)                                 # Векторизуем предобработанные тексты
covid_pred = clf4.predict(vectorized_text)                                               # Делаем предсказания с помощью обученного классификатора
a = (np.count_nonzero(covid_pred == 'sad')) * 100 / len(covid_pred)                      # Доля предсказаний, результатом которых стала метка 'sad'
print(f"Доля постов, отнесенных к классу sad, от общего количества постов, равна {a}%")  

Доля постов, отнесенных к классу sad, от общего количества постов, равна 27.41756206002223%


# ***Анализ тональности постов***

In [50]:
covid = pd.read_csv("/content/covid19_tweets.csv")
covid

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ᏉᎥ☻լꂅϮ,astroworld,wednesday addams as a disney princess keepin i...,2017-05-26 05:46:42,624.0,950.0,18775.0,False,2020-07-25 12:27:21,If I smelled the scent of hand sanitizers toda...,NaN,Twitter for iPhone,False
1,Tom Basile 🇺🇸,"New York, NY","Husband, Father, Columnist & Commentator. Auth...",2009-04-16 20:06:23,2253.0,1677.0,24.0,True,2020-07-25 12:27:17,Hey @Yankees @YankeesPR and @MLB - wouldn't it...,NaN,Twitter for Android,False
2,Time4fisticuffs,"Pewee Valley, KY",#Christian #Catholic #Conservative #Reagan #Re...,2009-02-28 18:57:41,9275.0,9525.0,7254.0,False,2020-07-25 12:27:14,@diane3443 @wdunlap @realDonaldTrump Trump nev...,['COVID19'],Twitter for Android,False
3,ethel mertz,Stuck in the Middle,#Browns #Indians #ClevelandProud #[]_[] #Cavs ...,2019-03-07 01:45:06,197.0,987.0,1488.0,False,2020-07-25 12:27:10,@brookbanktv The one gift #COVID19 has give me...,['COVID19'],Twitter for iPhone,False
4,DIPR-J&K,Jammu and Kashmir,🖊️Official Twitter handle of Department of Inf...,2017-02-12 06:45:15,101009.0,168.0,101.0,False,2020-07-25 12:27:08,25 July : Media Bulletin on Novel #CoronaVirus...,"['CoronaVirusUpdates', 'COVID19']",Twitter for Android,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95615,CLDALLAS,"Peachtree City, GA","Biologist, author and theologian who loves peo...",2019-11-28 14:45:47,132.0,229.0,1949.0,False,2020-08-09 05:00:21,Amen! Fake saints do the same thing! They put ...,NaN,Twitter for iPhone,False
95616,UN Human Rights Asia,"Bangkok, Thailand",@UNHumanRights' regional representation in Sou...,2015-10-13 06:40:27,19917.0,1108.0,221.0,True,2020-08-09 05:00:21,“Realizing the rights of indigenous people’s m...,['COVID19'],Hootsuite Inc.,False
95617,martin Johnson,"Georgia, USA","I'm an award-winning screenwriter, blogger and...",2012-07-06 23:22:22,2859.0,4999.0,8784.0,False,2020-08-09 05:00:19,Our peace and hope is not of this world. #Sun...,"['SundayMorning', 'COVID19', 'trump', 'unemplo...",Buffer,False
95618,Leann Hackman-Carty,"Calgary, Alberta","Creating a better, more disaster-resilient wor...",2017-05-14 12:47:26,576.0,1123.0,1778.0,False,2020-08-09 05:00:19,My disaster guides are available on Kindle. Ge...,NaN,PromoRepublic,False


In [51]:
covid = covid[["text"]]

In [53]:
cleaned_texts = []                                                           # Создаём пустой список, в который поместим очищенные тексты
stop_word = stopwords.words('english')                                       # Загрузка стоп-слов для английского языка
prefix = ('https')                                                           # Префикс для удаления
for i in range(covid.shape[0]):                                              # Создаём цикл, который проходит по каждому тексту из датасета
  a = 0                                                                      # Текущее слово из текста (изначально равно нулю)
  text = str(covid.text[i])                                              
  text = text.replace('…', '')                                               # Удаляем многоточие
  text = text.replace('\n', ' ')                                             # Удаляем символы '\n'
  text = text.replace('&amp', ' ')                                           # Удаляем символы '&amp'
  text = remove_emoji(text)                                                  # Убираем из текста смайлики и эмоджи с помощью написанной выше функции
  text = " ".join([x for x in text.split() if not x.startswith(("https"))])  # Убираем из текста ссылки
  if text[-1] == " ":                                                        # Если последний символ текста -- пробел, то
    text = text[:len(s) - 1]                                                 # удаляем его
  cleaned_texts.append(text)                                                

In [54]:
covid["full_clear"] = cleaned_texts

In [55]:
sentiments = []                                        # Создаём пустой список для добавления настроений 
messages = covid.full_clear[0:100]                     # Отбираем первые сто записей (так как долго выполняется)
for mes in tqdm(messages):                             # Создаём цикл, который проходит через каждый текст в списке
  blob = TextBlob(mes, analyzer=NaiveBayesAnalyzer())  # Анализируем настроение
  sentiments.append(blob.sentiment.classification)     # Добавляем результат в список

  0%|          | 0/100 [00:00<?, ?it/s]

In [56]:
a = 0                                                                         # Счётчик для 'позитивных' текстов
b = 0                                                                         # Счётчик для 'негативных' текстов
for i in range(len(sentiments)):                                              # Создаём цикл, который проходит по каждому настроению из списка, полученного выше
  if sentiments[i] == "pos":                                                  # Если настроение позитивное, то
    a += 1                                                                    # добавляем единицу к счётчику 'позитивных' текстов
  elif sentiments[i] == "neg":                                                # Если настроение негативное, то
    b += 1                                                                    # добавляем единицу к счётчику 'позитивных' текстов
print(f"Доля постов, отнесенных к классу pos, среди 100 постов, равна {a}%")  # Вывод доли текстов с положительным настроением
print(f"Доля постов, отнесенных к классу neg, среди 100 постов, равна {b}%")  # Вывод доли текстов с негативным настроением

Доля постов, отнесенных к классу pos, среди 100 постов, равна 67%
Доля постов, отнесенных к классу neg, среди 100 постов, равна 33%


In [57]:
tokenizer = RegexTokenizer()                             # Токенизатор из библиотеки Dostoevskiy
model = FastTextSocialNetworkModel(tokenizer=tokenizer)  # Модель
messages = covid.full_clear                              # Передаём тексты в переменную messages
results = model.predict(messages, k=2)                   # Делаем предсказания (2 настроения)
sentiments_dost = []                                     # Создаём пустой список
for message, sentiment in zip(messages, results):        # Создаём цикл, который проходит по каждой строке словаря
    sentiments_dost.append(sentiment)                    # Добавляем в список настроение

In [58]:
class_dost = []                                          # Создаём пустой список
for i in range(len(sentiments_dost)):                    # Создаём цикл, который проходит по каждому настроению из списка выше
  class_dost.append(list(sentiments_dost[i].keys())[0])  # Добавляем в список метку настроения
covid["class_dost"] = class_dost                         # Добавляем в датафрейм столбец с меткой настроения

In [59]:
sentiments = covid.class_dost                                                                           # Создаём список с метками настроений
neutral = 0                                                                                             # Счётчик для 'нейтральных' текстов
skip = 0                                                                                                # Счётчик для пропущенных текстов
positive = 0                                                                                            # Счётчик для 'позитивных' текстов
speech = 0                                                                                              # Счётчик для 'разговорных' текстов без эмоций 
negative = 0                                                                                            # Счётчик для 'негативных' текстов
for i in range(len(sentiments)):                                                                        # Создаём цикл, который проходит по каждому настроению из списка, полученного выше
  if sentiments[i] == "neutral":                                                                        # Если настроение нейтральное, то
    neutral += 1                                                                                        # добавляем единицу к счётчику 'нейтральных' текстов
  elif sentiments[i] == "skip":                                                                         # Если нейросеть пропустила данный текст, то
    skip += 1                                                                                           # добавляем единицу к счётчику пропущенных текстов
  elif sentiments[i] == "positive":                                                                     # Если настроение позитивное, то
    positive += 1                                                                                       # добавляем единицу к счётчику 'позитивных' текстов
  elif sentiments[i] == "speech":                                                                       # Если настроение безэмоциональное, то
    speech += 1                                                                                         # добавляем единицу к счётчику 'разговорных' текстов без эмоций 
  elif sentiments[i] == "negative":                                                                     # Если настроение негативное, то
    negative += 1                                                                                       # добавляем единицу к счётчику 'негативных' текстов
print(f"Доля текстов с нейтральным настроением равна {round(neutral / covid.shape[0], 5)} %")          
print(f"Доля пропущенных текстов равна {round(skip / covid.shape[0], 5)} %")                          
print(f"Доля текстов с позитивным настроением равна {round(positive / covid.shape[0], 5)} %")          
print(f"Доля текстов без эмоций равна {round(Decimal(speech / covid.shape[0]), 5)} %")                  
print(f"Доля текстов с негативным настроением равна {round(Decimal(negative / covid.shape[0]), 5)} %") 

Доля текстов с нейтральным настроением равна 0.99863 %
Доля пропущенных текстов равна 0.00113 %
Доля текстов с позитивным настроением равна 0.00017 %
Доля текстов без эмоций равна 0.00005 %
Доля текстов с негативным настроением равна 0.00002 %
